In [1]:
import json
import os
import sys
import cProfile
import pstats

import numpy as np
from PIL import Image
from sklearn.metrics import mean_squared_error

sys.path.append(r"../")
from ct_assist import transform, estimator

# Performance
In this Notebook, I will be performance testing various parts of the ct_assist library. This includes both speed and accuracy testing.

First getting the testing data ready:

In [2]:
image_coord_dict = dict()
image_coord_dict["00.jpg"] = np.array([[1470, 1430], [2333, 1355], [3247, 1781], [1935, 1952]])
image_coord_dict["01.jpg"] = np.array([[1495, 1552], [2219, 1589], [1843, 1969], [805, 1875]])
image_coord_dict["03.jpg"] = np.array([[1216, 1398], [2215, 1754], [3268, 1530], [2067, 1282]])   

def setup_vars():
    """Loads data for test_transform_image"""
    data_dir = r"./data/table"
    json_fp = os.path.join(data_dir, "anno.json")
    arr_fp = os.path.join(data_dir, "anno.npz")
    with open(json_fp, "r") as fp:
        mappings = json.load(fp)

    with np.load(arr_fp) as arrs:
        anno_dict = {img: {"heads": arrs[f"{prefix}heads"],
                           "feet": arrs[f"{prefix}feet"]}
                     for img, prefix in mappings.items()}
    
    for key, items in anno_dict.items():
        if key.endswith("02.jpg"):
            continue
        else:
            image_coords = image_coord_dict[key[-6:]]
        # feet and heads have been swapped in annotations
        reference = np.array([items["feet"], items["heads"]])
        height = 0.095  # m
        STD = 0.01  # m
        img = Image.open(key)
        yield (img, reference, height, STD, image_coords)

## Speed

In [3]:
generator = setup_vars()
pr = cProfile.Profile()
pr.enable()
coords_sq = (transform.fit_transform(*params)[:, :2] for params in generator)
y_pred = np.array([estimator.area(poly) for poly in coords_sq])
pr.disable()
pr.print_stats("tottime")

         11384616 function calls (11149408 primitive calls) in 16.680 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1106335    1.760    0.000    1.760    0.000 {built-in method numpy.array}
    90021    1.556    0.000    5.677    0.000 camera.py:894(spaceFromImage)
    30006    1.134    0.000    4.884    0.000 _distn_infrastructure.py:1774(logpdf)
    30012    1.070    0.000    2.788    0.000 spatial.py:107(_initCameraMatrix)
892214/657514    1.050    0.000    4.421    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    90021    1.049    0.000    2.313    0.000 projection.py:335(getRay)
   120052    0.578    0.000    0.578    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        3    0.549    0.183   16.616    5.539 statistic.py:68(metropolis)
    60012    0.373    0.000    0.495    0.000 numerictypes.py:545(_can_coerce_all)
   180042    0.337    0.000    1.113    0.000 spatial.py:166(sp

       15    0.000    0.000    0.000    0.000 sre_compile.py:276(_optimize_charset)
        9    0.000    0.000    0.000    0.000 blocks.py:236(mgr_locs)
       21    0.000    0.000    0.000    0.000 Image.py:3437(__getitem__)
       18    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:62(_path_split)
       18    0.000    0.000    0.000    0.000 cast.py:1179(maybe_castable)
        3    0.000    0.000    0.001    0.000 indexing.py:1078(_getitem_axis)
       21    0.000    0.000    0.000    0.000 {built-in method _thread.allocate_lock}
      154    0.000    0.000    0.002    0.000 re.py:232(compile)
       78    0.000    0.000    0.000    0.000 common.py:1600(_is_dtype_type)
       45    0.000    0.000    0.000    0.000 inspect.py:2843(replace)
       65    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:58(<listcomp>)
       18    0.000    0.000    0.000    0.000 TiffImagePlugin.py:685(load_rational)
        9    0.000    0.000    0.000

       12    0.000    0.000    0.000    0.000 TiffImagePlugin.py:603(__delitem__)
        3    0.000    0.000    0.000    0.000 {method 'copy' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 synchronize.py:50(__init__)
       15    0.000    0.000    0.000    0.000 base.py:5648(maybe_extract_name)
        7    0.000    0.000    0.000    0.000 TiffImagePlugin.py:635(_register_basic)
       30    0.000    0.000    0.000    0.000 {method 'copy' of 'dict' objects}
      135    0.000    0.000    0.000    0.000 inspect.py:2521(default)
       21    0.000    0.000    0.000    0.000 common.py:194(is_object_dtype)
        8    0.000    0.000    0.000    0.000 format.py:282(descr_to_dtype)
        3    0.000    0.000    0.002    0.001 inference.py:391(is_dataclass)
        6    0.000    0.000    0.000    0.000 common.py:218(asarray_tuplesafe)
       12    0.000    0.000    0.000    0.000 parameter_set.py:98(__init__)
        3    0.000    0.000    0.009    0.003 Image.py

        8    0.000    0.000    0.000    0.000 format.py:597(<listcomp>)
        3    0.000    0.000    0.000    0.000 function.py:107(validate_argmax_with_skipna)
        8    0.000    0.000    0.000    0.000 tokenize.py:219(__init__)
       12    0.000    0.000    0.000    0.000 common.py:905(is_datetime64_any_dtype)
        1    0.000    0.000    0.001    0.001 threading.py:834(start)
        3    0.000    0.000    0.000    0.000 common.py:566(is_string_dtype)
        3    0.000    0.000    0.000    0.000 common.py:97(is_bool_indexer)
        3    0.000    0.000    0.000    0.000 managers.py:683(_consolidate_check)
        9    0.000    0.000    0.000    0.000 {built-in method _imp.is_frozen}
        3    0.000    0.000    0.000    0.000 _weakrefset.py:26(__exit__)
        8    0.000    0.000    0.000    0.000 random.py:256(choice)
        3    0.000    0.000    0.000    0.000 blocks.py:255(make_block_same_class)
        1    0.000    0.000    0.000    0.000 std.py:101(create_mp_lock

        3    0.000    0.000    0.000    0.000 function.py:86(process_skipna)
        3    0.000    0.000    0.000    0.000 std.py:1086(__del__)
        9    0.000    0.000    0.000    0.000 {built-in method builtins.ord}
        3    0.000    0.000    0.000    0.000 interactiveshell.py:1276(user_global_ns)
        3    0.000    0.000    0.000    0.000 series.py:1519(keys)
        3    0.000    0.000    0.000    0.000 generic.py:3573(_set_is_copy)
       15    0.000    0.000    0.000    0.000 TiffImagePlugin.py:700(load_undefined)
        6    0.000    0.000    0.000    0.000 {built-in method _operator.add}
        3    0.000    0.000    0.000    0.000 {method 'sort' of 'list' objects}
        9    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:424(has_location)
        9    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:719(create_module)
        2    0.000    0.000    0.000    0.000 _collections_abc.py:252(__subclasshook__)
        3    0.000  

## Accuracy
As the "real" positions in images currently aren't known, in order to figure out how close CameraTransform is to "real" positions, we have to take a derivative of these positions. For the purposes of this assignment, I've chosen the area of the polygon, as this is an important variable in the use case.

The three images used for testing all feature a table with a area of `133` cm$^{2}$. 

In [4]:
y_true = np.repeat(1.33455, y_pred.size)

In [5]:
y_pred

array([1.40257472, 1.35575164, 1.72792425])

In [6]:
mean_squared_error(y_true, y_pred, squared=False)

0.23081029229926853

An RMSE of around 20 square centimeters is adequate. Especially the latter two images are taken and worse angles and are insufficiently labeled in comparison to `img_03.jpg`.